<a href="https://colab.research.google.com/github/Aleksandar12312/DogBreedsRecognizer/blob/main/DogBreedsNeural.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from torchvision.datasets import ImageFolder
from torchvision import transforms
from google.colab import drive
from torch.utils.data import DataLoader,random_split

drive.mount('/content/drive')

!tar -xvf "/content/drive/MyDrive/dataset/images.tar" -C /content/

train_path = "/content/Images"

print(os.listdir("/content"))

transform = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                     std=[0.229, 0.224, 0.225]),



    transforms.RandomRotation(degrees=10)

])
dataset = ImageFolder(root=train_path, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)#bez shuffle model uci redosled
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)





#import matplotlib.pyplot as plt
#import numpy as np

#images, labels = next(iter(train_loader))

#fig, axes = plt.subplots(1, 4, figsize=(12,3))
#for i in range(4):
    #img = images[i].permute(1,2,0).numpy()
    #img = np.clip(img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406]), 0, 1)
    #axes[i].imshow(img)
    #axes[i].set_title(dataset.classes[labels[i]])
    #axes[i].axis('off')
#plt.show()

import torch as torch
import torch.nn as nn
import torchvision.models as models

#model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
#print(model)

class DogBreedModel(nn.Module):
    def __init__(self, number_classes=120):
        super(DogBreedModel, self).__init__()

        #Resnet18
        self.base_model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

        #Zamrznemo sve slojeve osim zadnjeg
        for param in self.base_model.parameters():
            param.requires_grad = False


        #Promenimo poslednji sloj da odgovara broju rasa
        in_features = self.base_model.fc.in_features
        self.base_model.fc = nn.Linear(in_features, number_classes)

        for param in self.base_model.parameters(): #ubacujemo nakon nekoliko epoha(2)
          param.requires_grad = True

    def forward(self, x):
        return self.base_model(x)
model = DogBreedModel()
model.load_state_dict(torch.load("/content/drive/MyDrive/model_weights.pth"))#dodato nakon prve epohe

criterion=torch.nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(), lr=1e-3)

#prebacuje model na gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

#trening
epochs = 10
for epoch in range(epochs):

    model.train()#ukljucuje droput i korisni trenutni batch za normalizaciju
    train_loss = 0
    train_correct = 0
    train_total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)#racun loss function, koliko je model pogresio
        loss.backward()#racuna gradient, odnosno u kom smeru je pogresio
        optimizer.step()#uci

        # loss i tačnost
        train_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

    #validacija
    #model.eval()#iskljucuje dropout i koristi batch koji je nakupio tokom treninga
    #val_loss = 0
    #val_correct = 0
    #val_total = 0
    #with torch.no_grad():
        #for images, labels in test_loader:
            #images, labels = images.to(device), labels.to(device)
            #outputs = model(images)
            #loss = criterion(outputs, labels)

            #val_loss += loss.item() * images.size(0)#ukupni loss
            #_, predicted = outputs.max(1)#uzima index klase sa najvecom predikcijom( u predict)//u _ je vrednost
            #val_total += labels.size(0)#broj primera
            #val_correct += (predicted == labels).sum().item()#sum sabira svaki true/ item prebacuje iz tensor u broj
    torch.save(model.state_dict(), "/content/drive/MyDrive/model_weights.pth")


    print(f"Epoch {epoch+1}/{epochs}")
    print(f" Train Loss: {train_loss/train_total:.4f} | Accuracy: {100*train_correct/train_total:.2f}%")
    #print(f" Val Loss:   {val_loss/val_total:.4f}   | Accuracy: {100*val_correct/val_total:.2f}%")
